# Robótica Móvel - Trabalho Prático 1 (Exercícios 5-6)
---
**Nome Completo:** Daniel Terra Gomes

**Matrícula:** 2025702870 

**Programa:** Mestrando do PPGCC  
**Departamento:** Departamento de Ciência da Computação  
**Instituição:** ICEx-UFMG  
**E-mail:** danielterragomes@ufmg.br

**Data:** 11 de setembro de 2025

---

Este notebook implementa os exercícios 5-6 do TP1, focando em:
- Integração de sensores laser (Hokuyo)
- Transformação de dados de sensores para frame global
- Navegação autônoma básica
- Mapeamento incremental com dados de laser

**Atenção:** Use a cena `T1-ex5.ttt` que contém o robô Pioneer P3DX com sensor laser.